# Add Forecasts to `System`

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse add time series data to a `System` using [PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

For example, a `System` created by [parsing a MATPOWER file](../../notebook/PowerSystems_examples/parse_matpower.ipynb)
doesn't contain any time series data. So a user may want to add forecasts to the `System`
### Dependencies
Let's use the 5-bus dataset we parsed in the MATPOWER example

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSystems_examples", "parse_matpower.jl"))

Project SIIPExamples v0.0.1
Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [336ed68f] CSV v0.7.7
  [9961bab8] Cbc v0.7.0
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.21.6
  [becb17da] Feather v0.5.6
  [60bf3e95] GLPK v0.13.0
  [2cd47ed4] InfrastructureSystems v0.15.0
  [b6b21f68] Ipopt v0.6.3
  [2535ab7d] JSON2 v0.3.2
  [4076af6c] JuMP v0.21.3
  [98b081ad] Literate v2.5.1
  [47be7bcc] ORCA v0.4.0
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v1.5.8
  [5f7eddb3] PowerGraphics v0.5.1
  [e690365d] PowerSimulations v0.6.2
  [bcd98974] PowerSystems v0.25.0
  [9e3dc215] TimeSeries v0.18.0
  [f269a46b] TimeZones v1.3.1
  [0f1e0344] WebIO v0.8.14
  [ade2ca70] Dates 
  [56ddb016] Logging 
  [44cfe95a] Pkg 
  [9a3f8284] Random 
  [10745b16] Statistics 
  [8dfed614] Test 
┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/PkG3S/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format with data

System
======
System Units Base: SYSTEM_BASE
Base Power: 100.0
Base Frequency: 60.0

Components
==========
Num components: 24

8×3 DataFrame. Omitted printing of 2 columns
│ Row │ ConcreteType             │
│     │ String                   │
├─────┼──────────────────────────┤
│ 1   │ Area                     │
│ 2   │ Bus                      │
│ 3   │ Line                     │
│ 4   │ LoadZone                 │
│ 5   │ PhaseShiftingTransformer │
│ 6   │ PowerLoad                │
│ 7   │ RenewableDispatch        │
│ 8   │ ThermalStandard          │

Forecasts
=========
Resolution: 0 seconds
Horizon: 0
Initial Times: 
Interval: 0 seconds
Components with Forecasts: 0
Total Forecasts: 0

### Define pointers to time series files
For example, if we want to add a bunch of time series files, say one for each load and
one for each renewable generator, we need to define pointers to each .csv file containing
the time series in the following format (PowerSystems.jl also supports a CSV format for this file)

In [2]:
FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts")
fname = joinpath(FORECASTS_DIR, "timeseries_pointers_da.json")
open(fname, "r") do f
    for line in eachline(f)
        println(line)
    end
end

[
    {
        "simulation": "DAY_AHEAD",
        "category": "Generator",
        "component_name": "SolarBusC",
        "label": "get_max_active_power",
        "scaling_factor": 1.0,
        "data_file": "./gen/Renewable/PV/da_solar5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "category": "Generator",
        "component_name": "WindBusA",
        "label": "get_max_active_power",
        "scaling_factor": 1.0,
        "data_file": "./gen/Renewable/WIND/da_wind5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "category": "ElectricLoad",
        "component_name": "bus2",
        "label": "get_max_active_power",
        "scaling_factor": 1.0,
        "data_file": "./load/da_load5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "category": "ElectricLoad",
        "component_name": "bus3",
        "label": "get_max_active_power",
        "scaling_factor": 1.0,
        "data_file": "./load/da_load5.csv"
    },
    {
        "simulation": "DAY_AHE

### Read the pointers

In [3]:
ts_pointers = PowerSystems.IS.read_time_series_metadata(fname)

5-element Array{InfrastructureSystems.TimeseriesFileMetadata,1}:
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "Generator", "SolarBusC", "get_max_active_power", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/PkG3S/data/forecasts/5bus_ts/gen/Renewable/PV/da_solar5.csv", Float64[], "DeterministicInternal", nothing)
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "Generator", "WindBusA", "get_max_active_power", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/PkG3S/data/forecasts/5bus_ts/gen/Renewable/WIND/da_wind5.csv", Float64[], "DeterministicInternal", nothing)
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "ElectricLoad", "bus2", "get_max_active_power", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/PkG3S/data/forecasts/5bus_ts/load/da_load5.csv", Float64[], "DeterministicInternal", nothing)
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "ElectricLoad", "bus3", "get_max_active_power", 1.0, "/Users/cbarrows/.julia/packages/Powe

### Read and assign forecasts to `System` using the `ts_pointers` struct

In [4]:
add_forecasts!(sys, ts_pointers)
sys

System
======
System Units Base: SYSTEM_BASE
Base Power: 100.0
Base Frequency: 60.0

Components
==========
Num components: 24

8×3 DataFrame. Omitted printing of 2 columns
│ Row │ ConcreteType             │
│     │ String                   │
├─────┼──────────────────────────┤
│ 1   │ Area                     │
│ 2   │ Bus                      │
│ 3   │ Line                     │
│ 4   │ LoadZone                 │
│ 5   │ PhaseShiftingTransformer │
│ 6   │ PowerLoad                │
│ 7   │ RenewableDispatch        │
│ 8   │ ThermalStandard          │

Forecasts
=========
Resolution: 60 minutes
Horizon: 24
Initial Times: 2020-08-06T00:00:00
Interval: 0 seconds
Components with Forecasts: 5
Total Forecasts: 5

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*